In [ ]:
#import modules for analyzing ,plotting, and formatting
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import seaborn as sns


In [ ]:
merged_800_1500 = pd.read_csv("merged_800m_1500m_df.csv")

In [ ]:
#import sklearn and relevant packages
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

#separate datasets for the model
#here the model is split 67/33
m800_1500_train, m800_1500_test = train_test_split(merged_800_1500, test_size=0.33, random_state=0)

#gl means grade level
#for this model we will use 800/1500m times and grade level
gl_800_1500_train = m800_1500_train[['800 Meters', '1500 Meters', 'Grade Level']]
gl_800_1500_test = m800_1500_test[['800 Meters', '1500 Meters', 'Grade Level']]


In [ ]:
#here we separate the features and labels. The features used are the 800m times as well as the grade leve
# this calls for dropping the 1500m column for the features df
#next I select only the 1500m column for the label df
features_800_train = gl_800_1500_train.drop('1500 Meters', axis=1)
label_train = gl_800_1500_train['1500 Meters'].copy()

#same steps are repeated for the test df
features_800_test = gl_800_1500_test.drop('1500 Meters', axis=1)
label_test = gl_800_1500_test['1500 Meters'].copy()

In [ ]:
features_800_train

,800 Meters,Grade Level
1380,123.60,12th Grade
1287,151.10,10th Grade
1081,124.52,12th Grade
572,123.79,11th Grade
1775,133.60,11th Grade
...,...,...
1033,123.30,12th Grade
1731,131.30,11th Grade
763,155.00,9th Grade
835,172.20,9th Grade


In [ ]:
#regression models require numeric data, this means that we have to convert variables from a categorical feature to a binary one.
#grade level is converted to 0s and 1s
# get dummies turned all non-numeric to numeric.
dummy_800_train = pd.get_dummies(features_800_train)

#after converting the grade level column, we reindex the df
dummy_800_train = dummy_800_train.reindex(columns=dummy_800_train.columns, fill_value=0)
features_800_train = dummy_800_train[['800 Meters', 'Grade Level_9th Grade', 'Grade Level_10th Grade', 'Grade Level_11th Grade', 'Grade Level_12th Grade']]
features_800_train

#the same steps are repeated for the test df
dummy_800_test = pd.get_dummies(features_800_test)

dummy_800_test = dummy_800_test.reindex(columns=dummy_800_test.columns, fill_value=0)
features_800_test = dummy_800_test[['800 Meters', 'Grade Level_9th Grade', 'Grade Level_10th Grade', 'Grade Level_11th Grade', 'Grade Level_12th Grade']]
features_800_test

,800 Meters,Grade Level_9th Grade,Grade Level_10th Grade,Grade Level_11th Grade,Grade Level_12th Grade
568,137.30,0,1,0,0
1180,122.30,0,1,0,0
1119,128.13,0,0,0,1
1820,156.00,0,0,0,1
399,146.00,1,0,0,0
...,...,...,...,...,...
1880,161.00,0,1,0,0
1487,130.60,0,0,1,0
564,150.00,1,0,0,0
294,148.42,0,1,0,0


In [ ]:
#here I am selecting the runners with times between 59 and 60 seconds in the 800m
#next I am finding those same runners in the labeled data (1500m times)
#I do this as a simple measure so that I can compare the single test predicitions below, and get 
# a sense of which model is predicting closes to what we expect a runner to run. 

times = features_800_train[(features_800_train['800 Meters'] > 119) & (features_800_train['800 Meters'] < 120)]
label_train[times.index].mean()

255.7447222222222

In [ ]:
# I used a small definition from Intro to machine learning to display the scores from the cross val.
#this code will be used later to display the scores from cross val cv
def display_scores(scores):
  print('Scores:', scores)
  print('Mean:', scores.mean()),
  print('Standard Deviation:', scores.std())


Here we have the data processed and formatted to run multiple ML models.
These will be LinearRegression, Decision Tree Regression, and Forest Regression.

Our main measure of the data is the RMSE. This function compares the values of the labels to the predictions. We compare this value for all the training data and choose a model that works best before we use the test data.

In [ ]:
#after preparing the data we are ready to create the linear regression instance
# and train the model with out data

linreg = LinearRegression()
linreg.fit(features_800_train, label_train)

#here we are importing another regression model to test the difference,
# and checking is a more powerful model will produce better results.
#we fit this model and predict on our training data as we did before.

tree_reg = DecisionTreeRegressor()
tree_reg.fit(features_800_train, label_train)

#the most powerful model used in this project
forest_reg = RandomForestRegressor()
forest_reg.fit(features_800_train, label_train)

#we import the one more regression model which will be the most powerful
#the steps will be the same as before

forest_reg = RandomForestRegressor()
forest_reg.fit(features_800_train, label_train)

RandomForestRegressor()

After creating an instance for each regression model, we fit each of them on the training data. 

After each fit we predict on training data and compare the training labels to analyze the accuracy.


In [ ]:
#after training we check out model by predicting using out training data
#we compare out predictions on training data vs. the labels and study the difference
# RMSE is the Root Mean Square Error, which measures the difference between predicted values and the labels. 
# this measure is common and works well with regression models.

train_pred = linreg.predict(features_800_train)
lin_mse = mean_squared_error(label_train, train_pred)
lin_rmse = np.sqrt(lin_mse)
print('linreg RMSE: ', lin_rmse)

linreg RMSE:  14.54682161150026


In [ ]:
#predicting with the Decision Tree Regression 
tree_pred = tree_reg.predict(features_800_train)
tree_mse = mean_squared_error(label_train, tree_pred)
tree_rmse = np.sqrt(tree_mse)
print('decision tree RMSE: ', tree_rmse)

decision tree RMSE:  7.786857701352396


In [ ]:
#prediction with Forest Regression
forest_pred = forest_reg.predict(features_800_train)
forest_mse = mean_squared_error(label_train, forest_pred)
forest_rmse = np.sqrt(forest_mse)

print('forest RMSE:',  forest_rmse)

forest RMSE: 9.146394029828686


In [ ]:
#here is a small test using a runner with a 60 second 800m.
# the number 1 represents a grade level.
#using this we can run a simple test to see how acurate a single prediction is.
#the array corresponds to 800m time, 9th grade, 10th grade, 11th grade, and 12th grade (in this order).

test_arr = np.array([120,0,0,0,1])
test_arr = test_arr.reshape(1,-1)

#predicting a single result with linreg
test_pred_lin = linreg.predict(test_arr)
test_pred_lin

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([255.68008304])

In [ ]:
#here we run the same test as before, with a single sample, and compare how the decision tree predicts.
#although the decision tree has a lower RMSE than linreg, the prediction is futher off what we would expect.
# this shows that the decision tree likely is overfitting the data

#predicting a single result with Decison tree
test_pred_tree = tree_reg.predict(test_arr)
test_pred_tree

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeRegressor was fitted with feature names
  warnings.warn(


array([251.31])

In [ ]:
# predicting a single result with forest regression 
test_pred_forest = forest_reg.predict(test_arr)
test_pred_forest

/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


array([254.4003])

In [ ]:
# the same cross val is run using linear regression

lin_scores = cross_val_score(linreg, features_800_train, label_train, 
                         scoring='neg_mean_squared_error', cv=10)

#compared to the original linreg, the mean here is marginally better, but it is still insightful to check 
#the original values and compare.

lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [12.92322322 12.8017793  12.30115494 14.35071304 21.87252578 15.1711855
 12.62522304 13.17126945 13.63156169 14.68578445]
Mean: 14.353442040808574
Standard Deviation: 2.6624468673682364


In [ ]:
#here I import cross_val_score which will allow me to test the decision tree 
#and improve how it fits by taking small chunks of the data and training on each chunk
# this results in a better fitting.

tree_scores = cross_val_score(tree_reg, features_800_train, label_train, 
                         scoring='neg_mean_squared_error', cv=10)

# displaying the scors shows a larger mean
#this is expected because before, the data was being overfit
tree_rmse_scores = np.sqrt(-tree_scores)
display_scores(tree_rmse_scores)

Scores: [18.93653344 18.85636828 16.16145525 21.53373923 26.82513086 19.89432869
 16.08310095 17.93840563 17.68309069 19.14774099]
Mean: 19.30598940056982
Standard Deviation: 2.949396391548148


In [ ]:
#just as with the decision tree, randome forest was likely overfitting the training data
# when using crossval we see that the mean is a amost twice as bad
# still, this model produces the best results
#this is also expected because this model is more powerful 

forest_scores = cross_val_score(forest_reg, features_800_train, label_train, 
                         scoring='neg_mean_squared_error', cv=10)

forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores: [15.67033026 16.6354771  13.48735116 18.63764456 23.80039426 17.10687907
 13.77525981 15.11343507 15.26869911 16.4966374 ]
Mean: 16.59921078073425
Standard Deviation: 2.806791739475534


In [ ]:
#after going through the models, we import one last package, GridSearchCV
#this sorts through many parameters and returns the best one.

param_grid = [
    {'n_estimators': [3,10,30], 'max_features': [2,4,6,8]},
    {'bootstrap': [False], 'n_estimators': [3,10], 'max_features': [2,3,4]}
]
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

#here the grid search model is used to fit the training data
#next we print out the best params and best estimators
grid_search.fit(features_800_train, label_train)
print(grid_search.best_params_)
print(grid_search.best_estimator_)

{'max_features': 6, 'n_estimators': 30}
RandomForestRegressor(max_features=6, n_estimators=30)


In [ ]:
#here the features and estimators are zip together with the scores
#then they are printed out so we can see which returned the best and the range of values 
#that result from the changes in parameters

cv_res = grid_search.cv_results_

for mean_score, params in zip(cv_res['mean_test_score'], cv_res['params']):
  print(np.sqrt(-mean_score), params)

18.234187367183587 {'max_features': 2, 'n_estimators': 3}
17.344881530925655 {'max_features': 2, 'n_estimators': 10}
17.27677521962205 {'max_features': 2, 'n_estimators': 30}
17.869886401982072 {'max_features': 4, 'n_estimators': 3}
16.88103190666687 {'max_features': 4, 'n_estimators': 10}
16.854673269521033 {'max_features': 4, 'n_estimators': 30}
17.601764193034462 {'max_features': 6, 'n_estimators': 3}
17.10946149847384 {'max_features': 6, 'n_estimators': 10}
16.764509632028762 {'max_features': 6, 'n_estimators': 30}
17.989635579020593 {'max_features': 8, 'n_estimators': 3}
17.076785241758294 {'max_features': 8, 'n_estimators': 10}
16.882692220395192 {'max_features': 8, 'n_estimators': 30}
19.248165158237335 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
19.175622339132545 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
19.06505987701021 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
18.995490058782842 {'bootstrap': False, 'max_features': 3, 'n_est

In [ ]:
#lastly we isolate the best estimators from the model
#we then predict the TEST data and compare to the test labels
final_model = grid_search.best_estimator_
final_pred = final_model.predict(features_800_test)

final_mse = mean_squared_error(label_test, final_pred)
final_rmse = np.sqrt(final_mse)
print(final_rmse)

16.77809638885795


In [ ]:
test_pred = final_model.predict(test_arr)
print(test_pred)

[256.26533333]


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
